In [ ]:
import os
from glob import glob
from IPython.display import clear_output
import json
import numpy as np
from sklearn import preprocessing
from matplotlib import pyplot as plt
%matplotlib inline
from GPUtil import showUtilization as gpu_usage
from time import time
import re

from utils import read_graph_data, read_pickle, plot_histogram, process_graph_data

In [ ]:
from torch_geometric.data import Data, Dataset
class CFG(Dataset):
    
    def __init__(self, graph_addr_list, root, label_transformer, sample_desc = None, transform=None, pre_transform=None):
        self.graph_addr_list = graph_addr_list
        self.label_transformer = label_transformer
        self.sample_desc = sample_desc
        super(CFG, self).__init__(root, transform, pre_transform)

    @property
    def processed_file_names(self):
        return [f'data_{file_idx}.pt' for file_idx in range(len(self.graph_addr_list))]

    def process(self):
        graph_data_mapping = []
        one_hot_transform = T.OneHotDegree(max_degree = 187, cat = False, in_degree = True)

        for graph_idx, graph_addr in enumerate(self.graph_addr_list):
            try:
                graph_data = read_graph_data(graph_addr)

                raw_edges = graph_data['edge_list']
                raw_nodes = list(graph_data['node_dict'].keys())

                if 'benign' in graph_addr:
                    y = 'benign'
                elif 'tsunami' in graph_addr:
                    y = 'tsunami'
                elif 'mirai' in graph_addr:
                    y = 'mirai'
                elif 'gafgyt' in graph_addr:
                    y = 'gafgyt'
                    

                unique_node_idx_counter = 0
                node_mapping = {}
                edges = [[], []]

                for node in raw_nodes:
                    node_mapping[str(node)] = unique_node_idx_counter
                    unique_node_idx_counter += 1

                for edge in raw_edges:
                    edges[0].append(node_mapping[str(edge[0])])
                    edges[1].append(node_mapping[str(edge[1])])

                edge_idx = torch.tensor(edges, dtype=torch.long)
                y = torch.tensor(self.label_transformer.transform([y]), dtype=torch.long)
                x = None
                data = Data(x=x, edge_index=edge_idx, y=y)
                data = one_hot_transform(data)
                
                if self.pre_filter is not None and not self.pre_filter(data):
                    continue

                if self.pre_transform is not None:
                    data = self.pre_transform(data)

                torch.save(data, osp.join(self.processed_dir, 'data_{}.pt'.format(graph_idx)))
            except Exception as e:
                print(graph_addr, e)

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, 'data_{}.pt'.format(idx)))
        return data

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
gpu_usage()

In [ ]:
benign_train_graph_list = glob("datasets/normal_dataset/benign/train/*")
benign_test_graph_list = glob("datasets/normal_dataset/benign/test/*")
print('benign', len(benign_train_graph_list), len(benign_test_graph_list))

tsunami_train_graph_list = glob("datasets/normal_dataset/tsunami/train/*")
tsunami_test_graph_list = glob("datasets/normal_dataset/tsunami/test/*")
print('tsunami', len(tsunami_train_graph_list), len(tsunami_test_graph_list))

mirai_train_graph_list = glob("datasets/normal_dataset/mirai/train/*")
mirai_test_graph_list = glob("datasets/normal_dataset/mirai/test/*")
print('mirai', len(mirai_train_graph_list), len(mirai_test_graph_list))

gafgyt_train_graph_list = glob("datasets/normal_dataset/gafgyt/train/*")
gafgyt_test_graph_list = glob("datasets/normal_dataset/gafgyt/test/*")
print('gafgyt', len(gafgyt_train_graph_list), len(gafgyt_test_graph_list))

all_train_graphs_list = benign_train_graph_list + tsunami_train_graph_list + gafgyt_train_graph_list + mirai_train_graph_list
all_test_graphs_list = benign_test_graph_list + tsunami_test_graph_list + gafgyt_test_graph_list + mirai_test_graph_list

print(f'Train: {len(all_train_graphs_list)}, Test: {len(all_test_graphs_list)}')


for i in benign_train_graph_list + benign_test_graph_list + tsunami_train_graph_list + tsunami_test_graph_list + gafgyt_train_graph_list + gafgyt_test_graph_list + mirai_train_graph_list + mirai_test_graph_list:
    if not os.path.exists(i):
        print(i)

all_train_graphs_list = benign_train_graph_list + tsunami_train_graph_list + gafgyt_train_graph_list + mirai_train_graph_list
all_test_graphs_list = benign_test_graph_list + tsunami_test_graph_list + gafgyt_test_graph_list + mirai_test_graph_list

print(f'Train: {len(all_train_graphs_list)}, Test: {len(all_test_graphs_list)}')

In [ ]:
import os
import os.path as osp 
from torch_geometric.data import Data, Dataset, Batch

from torch_geometric.loader import DataLoader
from torch.nn import Linear, Sequential, ReLU
from torch_scatter import scatter_mean
import torch.nn.functional as F
from torch_geometric.nn import GraphConv, GCNConv, SAGEConv, GATConv, GINConv, global_mean_pool, global_add_pool
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.data import Data
import torch_geometric.transforms as T

In [ ]:
selected_classes = ['benign', 'tsunami', 'mirai', 'gafgyt']

le = preprocessing.LabelEncoder()
le.fit(selected_classes)

print('Label Encdoer: ', le.transform(['benign', 'tsunami', 'mirai', 'gafgyt']))


train_dataset = CFG(graph_addr_list = all_train_graphs_list, root='data/train', label_transformer = le)
print(f'Number of training graphs: {train_dataset.len()}')

test_dataset = CFG(graph_addr_list = all_test_graphs_list, root='data/test', label_transformer = le)
print(f'Number of test graphs: {test_dataset.len()}')

adversarial_dir = "datasets/adversarial_dataset"
adversarial_dataset = {'adversarial_benign': [], 'adversarial_gafgyt': [], 'adversarial_mirai': [], 'adversarial_tsunami': []}
for i in glob(adversarial_dir + '/*'):
    adversarial_graph_data = read_graph_data(i)
    if re.search(r"{}_(max|median|min)".format('benign'), i):
        adversarial_dataset['adversarial_benign'].append(T.OneHotDegree(max_degree = 187, cat = False, in_degree = True)(process_graph_data(adversarial_graph_data, 'adversarial', le)))
    elif re.search(r"{}_(max|median|min)".format('gafgyt'), i):
        adversarial_dataset['adversarial_gafgyt'].append(T.OneHotDegree(max_degree = 187, cat = False, in_degree = True)(process_graph_data(adversarial_graph_data, 'adversarial', le)))
    elif re.search(r"{}_(max|median|min)".format('mirai'), i):
        adversarial_dataset['adversarial_mirai'].append(T.OneHotDegree(max_degree = 187, cat = False, in_degree = True)(process_graph_data(adversarial_graph_data, 'adversarial', le)))
    elif re.search(r"{}_(max|median|min)".format('tsunami'), i):
        adversarial_dataset['adversarial_tsunami'].append(T.OneHotDegree(max_degree = 187, cat = False, in_degree = True)(process_graph_data(adversarial_graph_data, 'adversarial', le)))

print(f"Number of adversarial graphs: {len(adversarial_dataset['adversarial_benign']) + len(adversarial_dataset['adversarial_gafgyt']) + len(adversarial_dataset['adversarial_mirai']) + len(adversarial_dataset['adversarial_tsunami'])}")


In [ ]:
def split_dataset(train_dataset, test_dataset, adversarial_dataset, batch_size):
    dataset_dict = {
        'normal': {'train': DataLoader(train_dataset, batch_size = batch_size, shuffle = True), 'test': DataLoader(test_dataset, batch_size = batch_size, shuffle = False)},
        'adversarial': {adv_k: DataLoader(adv_v, batch_size = batch_size, shuffle = False) for adv_k, adv_v in adversarial_dataset.items()}
    }

    for k, v in dataset_dict.items():
        if k == 'adversarial':
            for temp_k, temp_v in v.items():
                print(f'Number of {temp_k} Samples: {len(temp_v.dataset)}')
        else:
            print(f"Number of {k} samples in train-set: {len(v['train'].dataset)}", f"Number of {k} samples in test-set: {len(v['test'].dataset)}")

    return dataset_dict

In [ ]:
class Detector(torch.nn.Module):
    def __init__(self, config):
        super(Detector, self).__init__()
        self.config = config

        if self.config['conv_layer_type'].__name__ == 'GCNConv':
            self.graphConv_layer_list = torch.nn.ModuleList([self.config['conv_layer_type'](*layer, bias = False) for layer in self.config['layer_size_list']])
        elif self.config['conv_layer_type' ].__name__ == 'GINConv':
            self.graphConv_layer_list = torch.nn.ModuleList()
            for layer in self.config['layer_size_list']:
                in_mlp = Sequential(
                    Linear(layer[0], layer[1], bias = False), 
                    torch.nn.BatchNorm1d(layer[1]), 
                    ReLU(), 
                    Linear(layer[1], layer[1], bias = False))
                self.graphConv_layer_list.append(self.config['conv_layer_type'](in_mlp))
        elif self.config['conv_layer_type'].__name__ == 'GATConv':
            self.graphConv_layer_list = torch.nn.ModuleList()
            for layer_idx, layer in enumerate(self.config['layer_size_list']):
                if layer_idx == 0:
                    self.graphConv_layer_list.append(self.config['conv_layer_type'](*layer, heads = 8, bias = False, dropout = config['dropout']))
                elif layer_idx < len(self.config['layer_size_list']) - 1:
                    self.graphConv_layer_list.append(self.config['conv_layer_type'](in_channels = layer[0] * 8, out_channels = layer[1], heads = 8, bias = False, dropout = config['dropout']))
                else:
                    self.graphConv_layer_list.append(self.config['conv_layer_type'](in_channels = layer[0] * 8, out_channels = layer[1], heads = 1, bias = False, dropout = config['dropout']))
        elif self.config['conv_layer_type'].__name__ == 'SAGEConv':
            self.graphConv_layer_list = torch.nn.ModuleList([self.config['conv_layer_type'](*layer, aggr = config['pooling_option'].__name__.split('_')[1], bias = False) for layer in self.config['layer_size_list']])

        if self.config['batch_normalization']:
            self.bn_list = torch.nn.ModuleList([torch.nn.BatchNorm1d(layer[1]) for layer in self.config['layer_size_list']])

        self.linear_layer_list = torch.nn.ModuleList([Linear(32, 32, bias = False), Linear(32, 32, bias = False)])
        self.linear_bn_list = torch.nn.ModuleList([torch.nn.BatchNorm1d(32)])

    def forward(self, x_data, edge_index_data, batch):
        x, edge_index = x_data, edge_index_data

        if self.config['virtual_edges']:
            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes = x.size(0))

        num_layers = len(self.graphConv_layer_list)
        for layer_idx, layer in enumerate(self.graphConv_layer_list):
            x = layer(x, edge_index)
            if self.config['conv_layer_type'].__name__ == 'GCNConv':
                if self.config['batch_normalization']:
                    x = self.bn_list[layer_idx](x)
                x = self.config['activation_func'](x)
                if self.config['dropout'] > 0.0 and self.training:
                    x = F.dropout(x, p = self.config['dropout'], training = self.training)
            elif self.config['conv_layer_type'].__name__ == 'GINConv':
                if self.config['batch_normalization']:
                    x = self.bn_list[layer_idx](x)
                x = self.config['activation_func'](x)
                if self.config['dropout'] > 0.0 and self.training: 
                    x = F.dropout(x, p = self.config['dropout'], training = self.training) 
            elif self.config['conv_layer_type'].__name__ == 'GATConv':
                if self.config['batch_normalization']:
                    x = self.bn_list[layer_idx](x)
                x = self.config['activation_func'](x)
                if self.config['dropout'] > 0.0 and self.training: 
                    x = F.dropout(x, p = self.config['dropout'], training = self.training)
            elif self.config['conv_layer_type'].__name__ == 'SAGEConv':
                if self.config['batch_normalization']:
                    x = self.bn_list[layer_idx](x)
                x = self.config['activation_func'](x)
                if self.config['dropout'] > 0.0 and self.training: 
                    x = F.dropout(x, p = self.config['dropout'], training = self.training)               

        x_pooled = self.config['pooling_option'](x, batch)

        num_layers = len(self.linear_layer_list)
        for layer_idx, layer in enumerate(self.linear_layer_list):
            x_pooled = layer(x_pooled)
            if layer_idx < num_layers - 1:
                if self.config['batch_normalization']:
                    x_pooled = self.linear_bn_list[layer_idx](x_pooled)
                x_pooled = self.config['activation_func'](x_pooled)
                if self.config['dropout'] > 0.0 and self.training:
                    x_pooled = F.dropout(x_pooled, p = self.config['dropout'], training = self.training)           
        
        return x, x_pooled

In [ ]:
def get_config_str(config):
    str_config = {}
    for k, v in config.items():
        if type(v).__name__ == 'function' or type(v).__name__ == 'type':
            str_config[k] = v.__name__
        elif type(v).__name__ == 'dict':
            for k1, v1 in v.items():
                if type(v1).__name__ == 'function' or type(v1).__name__ == 'type':
                    str_config[k1] = v1.__name__
                else:
                    str_config[k1] = v1
        else:
            str_config[k] = v
    str_config = str(str_config)

    return str_config

def get_model_setting(config):
    layer_size_str = ''.join(str(s[-1])+'-' for s in config['layer_size_list'])[:-1]
    conv_layer_type_str = config['conv_layer_type'].__name__
    batch_size_str = str(config['batch_size'])
    bn_str = 'batchNorm' if config['batch_normalization'] else 'noBatchNorm'
    pooling_str = config['pooling_option'].__name__.split('_')[1] + 'Pool'
    dropout_str = 'dropout' + str(config['dropout'])
    lr_str = 'lr' + str(config['lr'])
    nu_str = str(config['nu'])
    wd_lamda_str = str(config['wd_lambda'])
    eps_str = str(config['eps'])
    loss_type_str = config['loss_type']
    model_setting = f'batchSize{batch_size_str}_{conv_layer_type_str}{layer_size_str}_linear32-32_{pooling_str}_{bn_str}_{dropout_str}_{lr_str}_nu{nu_str}_wdL{wd_lamda_str}_eps{eps_str}_loss{loss_type_str}'
    return model_setting

In [ ]:
def train_test_encoder(config):
    save_addr_base = 'detector'
    EPOCHS = 2
    dataset_dict = split_dataset(train_dataset, test_dataset, adversarial_dataset, config['batch_size'])
    model_setting, config_str = get_model_setting(config), get_config_str(config)
    print(f'Model Setting is: {model_setting}')

    error_dict, score_dist, detection_dict = {}, {}, {}

    for normal_class in dataset_dict.keys():
        if normal_class == 'adversarial':
            continue
        
        print(f'<<<--- Normal Class is: {normal_class} --->>>')

        model = Detector(config).to(device)
        print(model)
        optimizer = torch.optim.Adam(model.parameters(), lr = config['lr'], weight_decay = config['wd_lambda'])

        data_center = None
        radius = torch.tensor(0, device = device)

        '****** START Data Center Measurement ******'
        print('Measuring Data Center.')
        model.eval()
        with torch.no_grad():
            for data_idx, data in enumerate(dataset_dict[normal_class]['train']):
                data_gpu = data.to(device)
                _, graph_embedding_batch = model(data_gpu.x, data_gpu.edge_index, data_gpu.batch)
                if data_idx == 0:
                    data_center = graph_embedding_batch
                else:
                    data_center = torch.cat((data_center, graph_embedding_batch), dim = 0)
            data_center = data_center.mean(dim = 0)
            data_center[(abs(data_center) < config['eps']) & (data_center < 0)] = -config['eps']
            data_center[(abs(data_center) < config['eps']) & (data_center > 0)] = config['eps']
        print('Done Measuring Data Center.')
        '****** END Data Center Measurement ******'

        error_dict[normal_class] = {'train': [], 'test': {'adversarial': {}}}
        for k in dataset_dict.keys():
            if k != 'adversarial':
                error_dict[normal_class]['test'][k] = []
            else:
                for err_k in dataset_dict[k].keys():
                    error_dict[normal_class]['test'][k][err_k] = []
        print('init error dict is: ', error_dict)
        score_dist[normal_class] = {}
        detection_dict[normal_class] = {}
        
        for epoch in range(EPOCHS):
            print(f'<<<<====== Epoch {epoch + 1} / {EPOCHS} ======>>>>')
            score_dist[normal_class][epoch] = {'train': [], 'test': {k: [] for k in dataset_dict.keys()}}
            score_dist[normal_class][epoch]['test']['adversarial'] = {k: [] for k in dataset_dict['adversarial'].keys()}


            "****** START Training Epoch ******"
            model.train()
            epoch_loss, n_batches = 0, 0
            for data_idx, data in enumerate(dataset_dict[normal_class]['train']):
                model.zero_grad()
                data_gpu = data.to(device)
                _, graph_embedding_batch = model(data_gpu.x, data_gpu.edge_index, data_gpu.batch)
                dist = torch.sum((graph_embedding_batch - data_center) ** 2, dim=1)

                if config['loss_type'] == 'soft_boundary':
                    scores = dist - radius ** 2
                    loss = radius ** 2 + (1 / config['nu']) * torch.mean(torch.max(torch.zeros_like(scores), scores))
                else:
                    scores = dist
                    loss = torch.mean(dist)

                score_dist[normal_class][epoch]['train'] = scores.detach().to('cpu').numpy() if data_idx == 0 else np.concatenate((score_dist[normal_class][epoch]['train'], scores.detach().to('cpu').numpy()), axis = 0)                

                loss.backward()
                optimizer.step()

                "****** START Measuring R ******"
                if config['loss_type'] == 'soft_boundary':
                    radius = np.quantile(np.sqrt(dist.detach().to('cpu').numpy()), 1 - config['nu'])
                "****** END Measuring R ******"

                n_batches += 1
                epoch_loss += loss.item()
            epoch_loss /= n_batches
            error_dict[normal_class]['train'].append(epoch_loss)
            score_dist[normal_class][epoch]['train'] = score_dist[normal_class][epoch]['train'].tolist()

            plt.figure(figsize=(15, 5))
            plt.plot(np.arange(1, epoch + 2, 1).tolist(), error_dict[normal_class]['train'])
            plt.grid()
            plt.title(f'Train Loss for {model_setting}')
            plt.xticks(range(1, epoch + 2))
            plt.savefig(f'{save_addr_base}/plots/{model_setting}_train_loss.pdf', bbox_inches='tight')
            plt.close()
            "****** END Training Epoch ******"

            "****** START Testing Epoch ******"
            model.eval()
            plt.figure(figsize=(15, 5))
            for class_name in dataset_dict.keys():
                epoch_loss, n_batches = 0, 0

                if class_name != 'adversarial':
                    loader = dataset_dict[class_name]['test']
                    with torch.no_grad():
                        for data_idx, data in enumerate(loader):
                            data_gpu = data.to(device)
                            _, graph_embedding_batch = model(data_gpu.x, data_gpu.edge_index, data_gpu.batch)

                            dist = torch.sum((graph_embedding_batch - data_center) ** 2, dim=1)

                            if config['loss_type'] == 'soft_boundary':
                                scores = dist - radius ** 2
                                loss = radius ** 2 + (1 / config['nu']) * torch.mean(torch.max(torch.zeros_like(scores), scores))
                            else:
                                scores = dist
                                loss = torch.mean(dist)
                        
                            score_dist[normal_class][epoch]['test'][class_name] = scores.detach().to('cpu').numpy() if data_idx == 0 else np.concatenate((score_dist[normal_class][epoch]['test'][class_name], scores.detach().to('cpu').numpy()), axis = 0)
            
                            n_batches += 1
                            epoch_loss += loss.item()
                        epoch_loss /= n_batches
                        error_dict[normal_class]['test'][class_name].append(epoch_loss)
                        score_dist[normal_class][epoch]['test'][class_name] = score_dist[normal_class][epoch]['test'][class_name].tolist()
            
                    plt.plot(np.arange(1, epoch + 2, 1).tolist(), error_dict[normal_class]['test'][class_name], label = class_name)

                if class_name == 'adversarial':
                    for sub_adv_k in dataset_dict['adversarial'].keys():
                        loader = dataset_dict['adversarial'][sub_adv_k]
                        with torch.no_grad():
                            for data_idx, data in enumerate(loader):
                                data_gpu = data.to(device)
                                _, graph_embedding_batch = model(data_gpu.x, data_gpu.edge_index, data_gpu.batch)

                                dist = torch.sum((graph_embedding_batch - data_center) ** 2, dim=1)

                                if config['loss_type'] == 'soft_boundary':
                                    scores = dist - radius ** 2
                                    loss = radius ** 2 + (1 / config['nu']) * torch.mean(torch.max(torch.zeros_like(scores), scores))
                                else:
                                    scores = dist
                                    loss = torch.mean(dist)
                            
                                score_dist[normal_class][epoch]['test'][class_name][sub_adv_k] = \
                                    scores.detach().to('cpu').numpy() if data_idx == 0 else np.concatenate((score_dist[normal_class][epoch]['test'][class_name][sub_adv_k], scores.detach().to('cpu').numpy()), axis = 0)
                
                                n_batches += 1
                                epoch_loss += loss.item()
                            epoch_loss /= n_batches
                            error_dict[normal_class]['test'][class_name][sub_adv_k].append(epoch_loss)
                            score_dist[normal_class][epoch]['test'][class_name][sub_adv_k] = score_dist[normal_class][epoch]['test'][class_name][sub_adv_k].tolist()
                
                        plt.plot(np.arange(1, epoch + 2, 1).tolist(), error_dict[normal_class]['test'][class_name][sub_adv_k], label = sub_adv_k)
            plt.grid()
            plt.title(f'Test Loss for {model_setting}')
            plt.legend()
            plt.xticks(range(1, epoch + 2))
            plt.savefig(f'{save_addr_base}/plots/{model_setting}_test_loss.pdf', bbox_inches='tight')
            plt.show()
            plt.close()

            print()


            for score_k, score_v in score_dist[normal_class][epoch]['test'].items():
                if score_k != 'adversarial':
                    print(f'Number of Scores for {score_k} Class: ', len(score_v), end = ' ')
                else:
                    for xx, yy in score_v.items():
                        print(f'Number of Scores for {xx} Class: ', len(yy), end = ' ')
            print()
    
            if config['loss_type'] == 'soft_boundary':
                normal_passed = np.count_nonzero(np.array(score_dist[normal_class][epoch]['test'][normal_class]) < 0)
                detection_dict[normal_class][epoch] = {'passed': normal_passed / len(score_dist[normal_class][epoch]['test'][normal_class]), 'detected': {}}
                print(f"# {normal_class} Samples Passed: {normal_passed / len(score_dist[normal_class][epoch]['test'][normal_class])} with a soft threshold of 0")

                for xx, yy in score_dist[normal_class][epoch]['test']['adversarial'].items():
                    adv_detected = np.count_nonzero(np.array(yy) > 0)
                    print(f"# {xx} Samples Detected: {adv_detected / len(yy)} with a soft threshold of 0")
                    detection_dict[normal_class][epoch]['detected'][xx] =  adv_detected / len(yy)
            elif config['loss_type'] == 'hard_boundary':
                prc_threshold = 95.9
                normal_pcn = np.percentile(score_dist[normal_class][epoch]['test'][normal_class], prc_threshold)
                detection_dict[normal_class][epoch] = {'passed': prc_threshold * 0.01, 'detected': {}}

                for xx, yy in score_dist[normal_class][epoch]['test']['adversarial'].items():
                    adv_detected = np.count_nonzero(np.array(yy) > normal_pcn)
                    print(f"# {xx} Samples Detected: {adv_detected / len(yy)} with a hard threshold of {normal_pcn}")
                    detection_dict[normal_class][epoch]['detected'][xx] =  adv_detected / len(yy)

 
            fig, ax = plt.subplots()
            fig.set_figheight(3);fig.set_figwidth(10)
            ax.set_title(f'The score Distribution with Normal Class: {normal_class}')
            plot_class_info = {}
            for score_k, score_v in score_dist[normal_class][epoch]['test'].items():
                if score_k != 'adversarial':
                    plot_class_info[score_k] = score_v
                else:
                    for adv_score_k, adv_score_v in score_v.items():
                        plot_class_info[adv_score_k] = adv_score_v
            ax.boxplot(plot_class_info.values())
            ax.set_xticklabels(plot_class_info.keys())
            ax.grid()
            plt.show()
            del plot_class_info
            "****** END Testing Epoch ******"

        
        with open(save_addr_base + f'/logs/' + f'score_dist_{model_setting}.json', 'w') as f:
            json.dump(score_dist[normal_class], f)
        with open(save_addr_base + f'/logs/' + f'error_{model_setting}.json', 'w') as f:
            json.dump(error_dict[normal_class], f)
        with open(save_addr_base + f'/logs/' + f'detection_{model_setting}.json', 'w') as f:
            json.dump(detection_dict[normal_class], f)

        clear_output()

In [ ]:
for config in [
  {'nu': 0.1, 'wd_lambda': 5e-4, 'eps': 0.001, 'lr': 0.001, 'loss_type': 'hard_boundary', 'batch_size': 128, 'dropout': 0.5, 'optimizer': torch.optim.Adam, 'virtual_edges': False, 'conv_layer_type': SAGEConv, 'layer_size_list': [(188, 64), (64, 64), (64, 32)], 'conv_normalize': False, 'batch_normalization': False, 'pooling_option': global_add_pool, 'activation_func': F.relu},
  ]:
  
  train_test_encoder(config)